In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 30.8 MB/s 
     |████████████████████████████████| 596 kB 55.7 MB/s 
     |████████████████████████████████| 6.6 MB 62.8 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import transformers
from transformers import BertTokenizerFast, RobertaTokenizerFast, TFEncoderDecoderModel, AdamWeightDecay
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from tensorflow.python.ops.numpy_ops import np_config
from pprint import pprint

# enable model saving at eager mode
np_config.enable_numpy_behavior()
print('transformers version:', transformers.__version__)
print('tensorflow version:', tf.__version__)

transformers version: 4.20.1
tensorflow version: 2.8.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_reviews = pd.read_json('/content/drive/MyDrive/Dataset/bbc_news_list_uk.json')


In [ ]:
df_reviews=df_reviews.head(10000)

In [ ]:
df_reviews

,tags,title,news_post_date,raw_content,content,url,author,language,_id,region,short_description,category,crawled_at
0,,Why quitting heroin substitute methadone is 'v...,2010-08-06T02:46:40.000Z,"<div data-component=""text-block"" class=""ssrcss...",The heroin substitute methadone can be used as...,https://www.bbc.co.uk/news/health-10869329,By Linda Pressly,en_GB,45cbcb62-840b-5a1d-9b59-d7f57f3ab7f6,Health,Heroin substitute methadone is used to wean ad...,BBC News,2021-05-02 17:19:39
1,"North Korea, Kim Jong-un, Kim Jong-il, Kim Jon...",North Korea leader's eldest son 'opposes dynasty',2010-10-12T08:54:42.000Z,"<div data-component=""text-block"" class=""ssrcss...",The eldest son of North Korean leader Kim Jong...,https://www.bbc.co.uk/news/world-asia-pacific-...,,en_GB,771b9c0d-88e1-589f-bf56-8ef80ce161a2,Asia-Pacific,The eldest son of North Korean leader Kim Jong...,BBC News,2021-05-02 17:19:39
2,,Paintings by gangster Reggie Kray go on sale i...,2010-10-27T17:35:24.000Z,"<div data-component=""text-block"" class=""ssrcss...",Seven oil paintings created by notorious gangs...,https://www.bbc.co.uk/news/uk-england-lincolns...,,en_GB,768e3fc6-c406-5570-8328-2e6fb75abd1b,Lincolnshire,A series of oil painting produced by gangster ...,BBC News,2021-05-02 17:19:39
3,,Bracklinn Falls Bridge hauled over gorge by hand,2010-10-08T13:21:50.000Z,"<div data-component=""text-block"" class=""ssrcss...",A 20-tonne bridge is being hauled into place b...,https://www.bbc.co.uk/news/uk-scotland-tayside...,,en_GB,a6e26396-70ec-501c-8ebc-1d600ddf6aab,Tayside and Central Scotland,A 20-tonne bridge is having to be hauled into ...,BBC News,2021-05-02 17:19:39
4,,Manchester soldier's last words become song,2010-09-22T17:35:34.000Z,"<div data-component=""text-block"" class=""ssrcss...",The final words written by a guardsman killed ...,https://www.bbc.co.uk/news/uk-england-manchest...,,en_GB,e8a150a5-b6e0-5315-9c04-fc639dee196f,Manchester,The final words written by a guardsman from Ma...,BBC News,2021-05-02 17:19:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15820,,Police hands over garage death inquiry to HSE,2010-07-20T00:45:18.000Z,"<div data-component=""text-block"" class=""ssrcss...",An investigation into the death of a young mec...,https://www.bbc.co.uk/news/uk-scotland-north-e...,,en_GB,ac717def-725f-5a4d-a19f-77ac6bb95771,"NE Scotland, Orkney & Shetland",An investigation into the death of a young mec...,BBC News,2021-05-02 18:08:38
15821,,Pringle appeals for help to create archive,2010-07-20T00:43:18.000Z,"<div data-component=""text-block"" class=""ssrcss...",One of the Borders' best known textile compani...,https://www.bbc.co.uk/news/uk-scotland-south-s...,,en_GB,0f761bbe-3b28-5d40-b40c-fe112260e8f1,South Scotland,Borders-based knitwear firm Pringle of Scotlan...,BBC News,2021-05-02 18:08:38
15822,,Voice technology 'could help detect autism',2010-07-19T23:09:28.000Z,"<div data-component=""text-block"" class=""ssrcss...",Young children with autism can be identified b...,https://www.bbc.co.uk/news/health-10686912,,en_GB,4631fe89-97cd-5d1d-b69b-6f16896bb242,Health,A new vocal analysis tool could help screen fo...,BBC News,2021-05-02 18:08:38
15823,,New homes target for Bristol and North Somerse...,2010-07-19T15:37:11.000Z,"<div data-component=""text-block"" class=""ssrcss...",Thousands of homes planned to be built in Bris...,https://www.bbc.co.uk/news/uk-england-bristol-...,,en_GB,f04abfa4-36b1-5a1c-8ee3-8cbc8921e609,Bristol,Thousands of homes planned for Bristol and Nor...,BBC News,2021-05-02 18:08:39


In [ ]:
df_reviews.drop_duplicates(subset=['content'], inplace=True)
# print(df_reviews[df_reviews['Summary'].isnull() == True]['Text'].unique())
df_reviews.dropna(subset=['short_description'], inplace=True)
df_reviews.reset_index(inplace=True)

In [ ]:
# Configure the training parameters
class TrainingConfig:
    val_split = 0.2
    pretrained_checkpoint = 'bert-base-uncased'
    encoder_checkpoint = 'bert-base-uncased'
    decoder_checkpoint = 'bert-base-uncased'
    pad_token_id = 0
    shared_weight = False
    encoder_max_len = 256 
    decoder_max_len = 30 
    nb_epoch = 3 
    learning_rate = 3e-5 
    batch_size = 8 
    
    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)

In [ ]:
# load the train and validation dataset
class DataLoader:
    def __init__(self, paragraphs, summaries, **kwargs):
        self.paragraphs = paragraphs 
        self.summaries = summaries 
        self.tokenizer = kwargs.get('tokenizer')
        self.val_split = kwargs.get('val_split')
        self.encoder_max_len = kwargs.get('encoder_max_len')
        self.decoder_max_len = kwargs.get('decoder_max_len')
    
    @property
    def sample_size(self):
        assert len(self.paragraphs)==len(self.summaries)
        return len(self.paragraphs)
    
    def split_train_test(self):
        train_idx, val_idx = train_test_split(
            list(range(self.sample_size)), 
            test_size=self.val_split, 
            random_state=98
        )
        return train_idx, val_idx
    def convert_text_to_ids(self, input_paragraphs, input_summaries):
        inputs = self.tokenizer(
            list(input_paragraphs), 
            return_tensors='np', 
            padding='max_length', 
            truncation=True, 
            max_length=self.encoder_max_len
        )
        outputs = self.tokenizer(
            list(input_summaries), 
            return_tensors='np', 
            padding='max_length', 
            truncation=True, 
            max_length=self.decoder_max_len
        )
        return inputs, outputs
    def list_to_tensor_dataset(self, input_paragraphs, input_summaries):
        inputs, outputs = self.convert_text_to_ids(
            input_paragraphs, 
            input_summaries
        )
        input_ids = tf.data.Dataset.from_tensor_slices(
            inputs['input_ids']
        )
        attention_masks = tf.data.Dataset.from_tensor_slices(
            inputs['attention_mask']
        )
        output_ids = tf.data.Dataset.from_tensor_slices(
            outputs['input_ids']
        )
        output_attention_masks = tf.data.Dataset.from_tensor_slices(
            outputs['attention_mask']
        )                                                
        tf_dataset = tf.data.Dataset.zip(
            ({
                'input_ids': input_ids, 
                'attention_mask': attention_masks,
                'decoder_input_ids': output_ids, 
                'decoder_attention_mask': output_attention_masks
            }, 
            output_ids)
        )
        return tf_dataset
    def __call__(self):
        train_idx, val_idx = self.split_train_test()
        train_paras, val_paras = self.paragraphs[train_idx], self.paragraphs[val_idx]
        train_sums, val_sums = self.summaries[train_idx], self.summaries[val_idx]
        train_dataset = self.list_to_tensor_dataset(train_paras, train_sums)
        val_dataset = self.list_to_tensor_dataset(val_paras, val_sums)
        return train_dataset, val_dataset

In [ ]:
# Customized loss function for seq2seq model
class Seq2SeqLoss(tf.keras.losses.Loss):
    def __init__(self, pad_token_id, name="seq2seq_loss"):
        super().__init__(name=name)
        self.pad_token_id = pad_token_id

    def call(self, y_true, y_pred):
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, 
            reduction=tf.keras.losses.Reduction.NONE
        )
        # shift the label and output sequences to match  
        output_logits = y_pred[:,:-1,:]
        input_labels = y_true[:,1:] 
        loss = loss_fn(input_labels, output_logits)
        # calculate loss without the padding tokens in label sequence
        mask = tf.cast((input_labels != self.pad_token_id), dtype=tf.float32)
        loss = loss * mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

In [ ]:
class Trainer:
    def __init__(self, model, loss_fn, optimizer, metric):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.metric = metric
        # loss tracker will capture the mean of loss till now
        self.loss_tracker = tf.keras.metrics.Mean(name='mean_loss')
    
    # Training Step
    @tf.function 
    def train_step(self, inputs):
        input_seqs, input_labels = inputs
        with tf.GradientTape() as tape: 
            outputs = self.model(
                input_seqs['input_ids'],
                input_seqs['attention_mask'],
                input_seqs['decoder_input_ids'],
                input_seqs['decoder_attention_mask'],
                training = True
            )
            logits = outputs.logits
            loss = self.loss_fn(input_labels, logits)
        gradients = tape.gradient(loss, self.model.trainable_weights)
        self.optimizer.apply_gradients(
            zip(gradients, self.model.trainable_weights)
        )
        self.loss_tracker.update_state(loss)
#         self.metric.update_state(y, predictions)
        return loss
    # Validation Step
    @tf.function  
    def val_step(self, inputs):
        input_seqs, input_labels = inputs
        outputs = self.model(                
                input_seqs['input_ids'],
                input_seqs['attention_mask'],
                input_seqs['decoder_input_ids'],
                input_seqs['decoder_attention_mask'],
                training = False
        )
        logits = outputs.logits
        loss = self.loss_fn(input_labels, logits)
        self.loss_tracker.update_state(loss)
#         self.metric.update_state(y,predictions)
        return loss

In [ ]:
def batched_generate_summary(model, tokenizer, batched_input):
    input_seqs, input_labels = batched_input
    outputs = model.generate(
        input_ids=input_seqs['input_ids'], 
        attention_mask=input_seqs['attention_mask']
    )
    output_strs = tokenizer.batch_decode(
        outputs, 
        skip_special_tokens=True
    )
    output_gold = tokenizer.batch_decode(
        input_seqs['decoder_input_ids'], 
        skip_special_tokens=True
    )
    input_strs = tokenizer.batch_decode(
        input_seqs['input_ids'], 
        skip_special_tokens=True
    )
    return output_strs, output_gold, input_strs

In [ ]:
reviews = df_reviews['content'].values
summaries = df_reviews['short_description'].values

training_config = TrainingConfig(nb_epoch=5)
tokenizer = BertTokenizerFast.from_pretrained(training_config.encoder_checkpoint)

dataloader_args = {
    'tokenizer': tokenizer,
    'val_split': training_config.val_split,
    'encoder_max_len': training_config.encoder_max_len,
    'decoder_max_len': training_config.decoder_max_len
}
dataloader = DataLoader(reviews, summaries, **dataloader_args)
train_dataset, val_dataset = dataloader()
train_dataset = (train_dataset
                 .shuffle(int(dataloader.sample_size*(1-training_config.val_split)))
                 .batch(training_config.batch_size))
val_dataset = val_dataset.batch(training_config.batch_size)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
bert2bert = TFEncoderDecoderModel.from_encoder_decoder_pretrained(
    training_config.encoder_checkpoint, 
    training_config.decoder_checkpoint,
    # whether to share the encoder weight
    tie_encoder_decoder=training_config.shared_weight
)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
All model checkpoint layers were used when initializing TFBertLMHeadModel.

Some layers of TFBertLMHeadModel were not initialized from the model checkpoint at bert

In [ ]:
bert2bert.save_pretrained('bert2bert')
bert2bert = TFEncoderDecoderModel.from_pretrained('bert2bert')

All model checkpoint layers were used when initializing TFEncoderDecoderModel.

All the layers of TFEncoderDecoderModel were initialized from the model checkpoint at bert2bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFEncoderDecoderModel for predictions without further training.


In [ ]:
# The special tokens for decoder should be aligned with the special tokens for encoder
# Since we are using Bert checkpoint for both decoder and decoder, 
# the cls and sep tokens in the encoder could be used as the start and end token for the decoder
bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id # 101
bert2bert.config.eos_token_id = tokenizer.sep_token_id # 102 
bert2bert.config.pad_token_id = tokenizer.pad_token_id # 0
bert2bert.config.vocab_size = bert2bert.config.encoder.vocab_size 

# These configurations are for the beam search in decoding process
bert2bert.config.max_length = 30
bert2bert.config.min_length = 3
bert2bert.config.no_repeat_ngram_size = 2
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4

In [ ]:
!pip install datasets
import datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 28.8 MB/s 
     |████████████████████████████████| 1.1 MB 61.7 MB/s 
     |████████████████████████████████| 212 kB 71.1 MB/s 
     |████████████████████████████████| 140 kB 72.2 MB/s 
     |████████████████████████████████| 127 kB 74.0 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 144 kB 43.6 MB/s 
     |████████████████████████████████| 271 kB 75.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
rouge = datasets.load_metric('rouge')
tf.keras.backend.clear_session()
trainer = Trainer(model=bert2bert,
                  loss_fn=Seq2SeqLoss(training_config.pad_token_id),
                  optimizer=AdamWeightDecay(
                      learning_rate=training_config.learning_rate, 
                      weight_decay_rate=0.005
                  ),
                  metric=None)

# Training Loop
for epoch in range(training_config.nb_epoch):
    print(f'\nEpoch {epoch+1}\n')
    print('Training....')
    for step,batched_input in enumerate(tqdm(train_dataset)):
        loss = trainer.train_step(batched_input)
        till_now_loss = trainer.loss_tracker.result()
        if step%200 == 0:
            print(f'Training loss for one batch at step {step}: {round(till_now_loss,3)}') 
    trainer.loss_tracker.reset_states()
    print('Validating....')
    val_measures = {'rouge precision':0, 'rouge recall':0, 'rouge f1': 0}
    for step, batched_input in enumerate(tqdm(val_dataset)):
        val_loss = trainer.val_step(batched_input)
#         pred_str, gold_str = generate_summary(bert2bert, 
#                                               tokenizer, 
#                                               batched_input)
#         rouge_output = rouge.compute(predictions=pred_str,
#                                      references=gold_str,
#                                      rouge_types=['rouge2'])['rouge2'].mid
#         val_measures['rouge precision'] += rouge_output.precision / len(val_dataset)
#         val_measures['rouge recall'] += rouge_output.recall / len(val_dataset)
#         val_measures['rouge f1'] += rouge_output.fmeasure / len(val_dataset)
    till_now_val_loss = trainer.loss_tracker.result()
    print(f'Validation loss: {round(till_now_val_loss,3)}')
    bert2bert.save_pretrained(
        f'bert2bert-Checkpoint-epoch{epoch+1}-loss{round(till_now_val_loss,3)}'
    )
#     for name, value in val_measures.items():
#         print(f'Validation {name}: {value}')
    trainer.loss_tracker.reset_states()


Epoch 1

Training....


  0%|          | 0/1546 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# Load the best model checkpoint
trained_bert2bert = TFEncoderDecoderModel.from_pretrained(
    './bert2bert-Checkpoint-epoch5-loss1.75600004196167'
)

OSError: ignored

In [ ]:
trained_bert2bert = TFEncoderDecoderModel.from_pretrained(
    '/content/drive/MyDrive/Dataset/bert2bert-Checkpoint-epoch4-loss0.40400001406669617'
)

All model checkpoint layers were used when initializing TFEncoderDecoderModel.

All the layers of TFEncoderDecoderModel were initialized from the model checkpoint at /content/drive/MyDrive/Dataset/bert2bert-Checkpoint-epoch4-loss0.40400001406669617.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFEncoderDecoderModel for predictions without further training.


In [ ]:
for step, batched_input in enumerate(tqdm(val_dataset)):
    pred_str, gold_str, input_strs = batched_generate_summary(
        trained_bert2bert, 
        tokenizer, 
        batched_input
    )
    rouge_output = rouge.compute(
        predictions=pred_str,
        references=gold_str,
        rouge_types=["rouge1"]
    )
    print('Rouge report: ')
    print(rouge_output['rouge1'].mid)
    for p_str,g_str,in_str in zip(pred_str, gold_str, input_strs):
        print('='*100)
        print('Review: ' + in_str)
        print('Summary: ' + g_str)
        print('Generated: ' + p_str)
    
    break

  0%|          | 0/387 [00:00<?, ?it/s]

Rouge report: 
Score(precision=0.3544728708791209, recall=0.213589479352347, fmeasure=0.26197916666666665)
Review: a man has been arrested for the second time after two sex attacks in an area of greater manchester. police are linking the sexual assault of a 20 - year - old woman on 1 july with a previous sex attack on a 16 - year - old girl nine days earlier in newton heath. a 20 - year - old man was arrested then bailed after the first attack but he has now been re - arrested following the second sexual assault. police appealed for information in connection to the attacks. in the first assault the teenage victim was attacked near the soap box pub in culceth lane, newton heath. in the second, the woman had been on her way to meet friend near st wilfred's school at 2230 bst on thursday. her attacker grabbed her from behind before sexually assaulting her as she screamed for help. det insp nick hughes said : " this man did not think twice about targeting a woman out alone and must be caug

In [ ]:
for step, batched_input in enumerate(tqdm(val_dataset)):
    pred_str, gold_str, input_strs = batched_generate_summary(
        trained_bert2bert, 
        tokenizer, 
        batched_input
    )
    rouge_output = rouge.compute(
        predictions=pred_str,
        references=gold_str,
        rouge_types=["rouge2"]
    )
    print('Rouge report: ')
    print(rouge_output['rouge2'].mid)
    for p_str,g_str,in_str in zip(pred_str, gold_str, input_strs):
        print('='*100)
        print('Review: ' + in_str)
        print('Summary: ' + g_str)
        print('Generated: ' + p_str)
    
    break

  0%|          | 0/387 [00:00<?, ?it/s]

Rouge report: 
Score(precision=0.19296085858585857, recall=0.11531033466017986, fmeasure=0.1419148785996612)
Review: a man has been arrested for the second time after two sex attacks in an area of greater manchester. police are linking the sexual assault of a 20 - year - old woman on 1 july with a previous sex attack on a 16 - year - old girl nine days earlier in newton heath. a 20 - year - old man was arrested then bailed after the first attack but he has now been re - arrested following the second sexual assault. police appealed for information in connection to the attacks. in the first assault the teenage victim was attacked near the soap box pub in culceth lane, newton heath. in the second, the woman had been on her way to meet friend near st wilfred's school at 2230 bst on thursday. her attacker grabbed her from behind before sexually assaulting her as she screamed for help. det insp nick hughes said : " this man did not think twice about targeting a woman out alone and must be ca

In [ ]:
for step, batched_input in enumerate(tqdm(val_dataset)):
    pred_str, gold_str, input_strs = batched_generate_summary(
        trained_bert2bert, 
        tokenizer, 
        batched_input
    )
    rouge_output = rouge.compute(
        predictions=pred_str,
        references=gold_str,
        rouge_types=["rougeL"]
    )
    print('Rouge report: ')
    print(rouge_output['rougeL'].mid)
    for p_str,g_str,in_str in zip(pred_str, gold_str, input_strs):
        print('='*100)
        print('Review: ' + in_str)
        print('Summary: ' + g_str)
        print('Generated: ' + p_str)
    
    break

  0%|          | 0/387 [00:00<?, ?it/s]

Rouge report: 
Score(precision=0.33478899572649573, recall=0.20634608957219253, fmeasure=0.2518366228070175)
Review: a man has been arrested for the second time after two sex attacks in an area of greater manchester. police are linking the sexual assault of a 20 - year - old woman on 1 july with a previous sex attack on a 16 - year - old girl nine days earlier in newton heath. a 20 - year - old man was arrested then bailed after the first attack but he has now been re - arrested following the second sexual assault. police appealed for information in connection to the attacks. in the first assault the teenage victim was attacked near the soap box pub in culceth lane, newton heath. in the second, the woman had been on her way to meet friend near st wilfred's school at 2230 bst on thursday. her attacker grabbed her from behind before sexually assaulting her as she screamed for help. det insp nick hughes said : " this man did not think twice about targeting a woman out alone and must be ca